In [ ]:
epochs = 50
size = 256

In [ ]:
!pip install gdal patchify tensorflow_addons geojson shapely geopandas rasterio==1.2.7 fiona
"> pipinstallation.txt 2>&1"
import rasterio
import os
from tensorflow.keras.applications import VGG16, InceptionResNetV2, ResNet50, Xception, InceptionV3
import rasterio.plot
import geopandas as gpd
import rasterio.mask
import random
import keras
import matplotlib.pyplot as plt
import numpy as np
import shapely
import gdal
import json
import tensorflow as tf
import skimage
import patchify
import rasterio.plot
import gc
import random
from keras.preprocessing.image import ImageDataGenerator
from sklearn import utils
from keras.models import Model, load_model
from keras.layers import Input, ZeroPadding2D
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, Model, Input
from sklearn.model_selection import train_test_split

In [ ]:
def readimage():
  image = gdal.Open("IMAGE PATH").ReadAsArray()
  image = rasterio.plot.reshape_as_image(image)
  return image
image = readimage()
image = image.astype('uint8')
image = image[:,:,0:3]
print(image.shape)

In [ ]:
def readgeo(link):
  geo = gpd.read_file(link)
  print(geo.crs)
  with rasterio.open("IMAGE PATH") as src:
    print(src.crs)
    raster = src.read()
    geo = geo.to_crs(src.crs)
    geo.plot()
    out_image, out_transform = rasterio.mask.mask(src, geo.geometry, filled=True)
    masks = out_image[3,:,:]
    masks[np.where(masks<=0)] = 0
    masks[np.where(masks>0)] = 1
    masks = masks.astype(np.int8)
    print(masks.shape)
    dolo_pixels = str(np.unique(masks, return_counts=True)[1][1])
    return masks, dolo_pixels
    
masks, dolo_pixels = readgeo("DOLOSTONE SHAPEFILE")
masks_limestone, limestone_pixels = readgeo("LIMESTONE SHAPEFILE")
masks_background, background_pixels = readgeo("HIGH VEGETATION SHAPEFILE")

In [ ]:
def combining():
  a = np.expand_dims(masks, axis=2)
  print(a.shape)
  b = np.expand_dims(masks_limestone, axis=2)
  print(b.shape)
  c = np.expand_dims(masks_background, axis=2)
  print(c.shape)
  y_train = np.concatenate((a,b,c), axis=2)
  return y_train
  
y_train = combining()
print(y_train.shape)
y_train = y_train.astype('uint8')

In [ ]:
y_train = skimage.util.view_as_windows(y_train, (size, size, 3), step = int(size/1.2))
print(y_train.shape)
y_train = y_train.reshape((-1,) + (size, size, 3))
print(y_train.shape)

def tiling():
  patch_arr = skimage.util.view_as_windows(image, (size, size, 3), step = int(size/1.2))
  print(patch_arr.shape)
  output = patch_arr.reshape((-1,) + (size, size, 3))
  print(output.shape)
  return output
image = tiling()
print(image.shape)

In [ ]:
for i in random.sample(range(0, y_train.shape[0]), 1):
  fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(5,5))
  ax1.imshow(y_train[i, :, :, 0], vmin=0, vmax=1)
  ax2.imshow(y_train[i, :, :, 1], vmin=0, vmax=1)
  ax3.imshow(y_train[i, :, :, 2], vmin=0, vmax=1)
  ax4.imshow(image[i,:,:,:])

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(image, y_train, test_size=0.1, shuffle=True, random_state=123)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
#undersampling
indexes = []
for i in np.arange(0, len(y_train)):
  if np.sum(y_train[i,:,:,2]) >= int(size * size * 0.99):
    indexes.append(i)
  else:
    continue

print(len(indexes))
import random
random.shuffle(indexes)
indexes = indexes[: int(len(indexes) / 2)]
print(len(indexes))
original = np.arange(0, len(y_train))
revised = np.delete(original, indexes)
print(len(revised))
y_train = y_train[revised, :, :, :]
print(y_train.shape)
x_train = x_train[revised, :, :, :]
print(x_train.shape)
'''select all images with 99% pixel tile as 1 on mask 2. 
random shuffle and select first half. 
delete these indexes from original list'''

#Oversampling lithology boundaries for dolomite
indexes = []
for i in np.arange(0, len(y_train)):
  if np.sum(y_train[i,:,:,0]) >= int(size * size * 0.33):
    indexes.append(i)
  else:
    continue
print(len(indexes))


random.shuffle(indexes)
indexes = indexes[: int(len(indexes) / 10)]
original = np.arange(0, len(y_train))
aa = y_train[indexes, :, :, :]
y_train = np.concatenate((aa, y_train), axis = 0)
bb = x_train[indexes, :, :, :]
x_train = np.concatenate((bb, x_train), axis = 0)

In [ ]:
datagen = ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip = True,
    rotation_range = 45,
    shear_range=25,
    brightness_range=(0.5, 1.5),
    validation_split=0.1)

In [ ]:
def iou_loss(y_true, y_pred):
      y_true = tf.reshape(y_true, [-1])
      y_pred = tf.reshape(y_pred, [-1])
      intersection = tf.reduce_sum(tf.cast(y_true, tf.float32) * tf.cast(y_pred, tf.float32))
      score = (intersection + 1.) / (tf.reduce_sum(tf.cast(y_true, tf.float32)) + 
      tf.reduce_sum(tf.cast(y_pred, tf.float32)) - intersection + 1.)
      return 1 - score

In [ ]:
train_generator = datagen.flow(x_train, y_train, batch_size=16, subset='training')
validation_generator = datagen.flow(x_train, y_train, batch_size=16, subset='validation')

In [ ]:
def custom_f1(y_true, y_pred):    
    def recall_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        
        recall = TP / (Positives+K.epsilon())    
        return recall 
    
    def precision_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Pred_Positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    
        precision = TP / (Pred_Positives+K.epsilon())
        return precision 
    
    precision, recall = precision_m(y_true, y_pred), recall_m(y_true, y_pred)
    
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)

In [ ]:
'''Xception'''
def conv_block(inputs, num_filters):
  s = Lambda(lambda x: x / 255) (inputs)
  x = Conv2D(num_filters, 2, padding='same')(s)
  x = BatchNormalization()(x)
  x = Dropout(0.2)(x)
  x = Activation('elu')(x)

  x = Conv2D(num_filters, 2, padding='same')(x)
  x = BatchNormalization()(x)
  x = Dropout(0.2)(x)
  x = Activation('elu')(x)
  
  return x

def decoder(inputs, skip_features, num_filters):
  x = Conv2DTranspose(num_filters, (2,2), strides=2, padding='same')(inputs)
  x = Concatenate()([x, skip_features])
  x = conv_block(x, num_filters)
  return x

def build_xception_unit(input_shape, size):
  '''Input'''
  inputs = Input(input_shape)
  '''Pretrained model'''
  xception = Xception(include_top = False, weights = 'imagenet', input_tensor = inputs)

  '''encoder'''
  s1 = xception.get_layer("input_1").output ##256

  s2 = xception.get_layer("block1_conv1_act").output ##127
  s2 = ZeroPadding2D(( (1, 0), (1, 0) ))(s2)  #128

  s3 = xception.get_layer("block3_sepconv2_bn").output ##63
  s3 = ZeroPadding2D(( (1, 0), (1, 0) ))(s3)  #64

  s4 = xception.get_layer("block4_sepconv2_bn").output ##32
  '''bridge'''
  b1 = xception.get_layer("block13_sepconv2_bn").output ##16
  '''decoder'''
  d1 = decoder(b1, s4, int(size/1))
  d2 = decoder(d1, s3, int(size/2))
  d3 = decoder(d2, s2, int(size/4))
  d4 = decoder(d3, s1, int(size/8))
  '''outputs'''
  outputs = Conv2D(3, (1,1), padding='same', activation='softmax')(d4)

  model = Model(inputs, outputs)
  return model


if __name__ == "__main__":
  input_shape = (size, size, 3)
  model = build_xception_unit(input_shape, size)
  model.summary()
model.compile(optimizer= opt, loss= iou_loss, metrics = [custom_f1, tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

history_4 = model.fit(train_generator,
                    validation_data = validation_generator,
                    steps_per_epoch=train_generator.n//train_generator.batch_size, 
                    validation_steps = validation_generator.n//validation_generator.batch_size,
                              epochs=epochs, verbose=1)

model.save('SAVE PATH')

results_4 = model.evaluate(x_test, y_test, batch_size=32)
print(results_4)

hist_df = pd.DataFrame(history_4.history)
hist_csv_file = 'HISTORY PATH'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

In [ ]:
'''VGG16'''
def conv_block(inputs, num_filters):
  s = Lambda(lambda x: x / 255) (inputs)
  x = Conv2D(num_filters, 3, padding='same')(s)
  x = BatchNormalization()(x)
  x = Dropout(0.2)(x)
  x = Activation('elu')(x)

  x = Conv2D(num_filters, 3, padding='same')(x)
  x = BatchNormalization()(x)
  x = Dropout(0.25)(x)
  x = Activation('elu')(x)
  
  return x

def decoder(inputs, skip_features, num_filters):
  x = Conv2DTranspose(num_filters, (2,2), strides=2, padding='same')(inputs)
  x = Concatenate()([x, skip_features])
  x = conv_block(x, num_filters)
  return x

def build_vgg16_unit(input_shape, size):
  '''Input'''
  inputs = Input(input_shape)
  '''Pretrained model'''
  vgg16 = VGG16(include_top = False, weights = 'imagenet', input_tensor = inputs)
  '''encoder'''
  s1 = vgg16.get_layer("block1_conv2").output ##512
  s2 = vgg16.get_layer("block2_conv2").output ##256
  s3 = vgg16.get_layer("block3_conv3").output ##128
  s4 = vgg16.get_layer("block4_conv3").output ##64
  '''bridge'''
  b1 = vgg16.get_layer("block5_conv3").output ##64
  '''decoder'''
  d1 = decoder(b1, s4, int(size/1))
  d2 = decoder(d1, s3, int(size/2))
  d3 = decoder(d2, s2, int(size/4))
  d4 = decoder(d3, s1, int(size/8))
  '''outputs'''
  outputs = Conv2D(3, (1,1), padding='same', activation='softmax')(d4)

  model = Model(inputs, outputs)
  return model


if __name__ == "__main__":
  input_shape = (size, size, 3)
  model = build_vgg16_unit(input_shape, size)
  model.summary()
model.compile(optimizer= opt, loss= iou_loss, metrics = [custom_f1, tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
history_3 = model.fit(train_generator,
                    validation_data = validation_generator,
                    steps_per_epoch=train_generator.n//train_generator.batch_size, 
                    validation_steps = validation_generator.n//validation_generator.batch_size,
                              epochs=epochs, verbose=1)
model.save('SAVE PATH')

results_3 = model.evaluate(x_test, y_test, batch_size=32)
print(results_3)

hist_df = pd.DataFrame(history_3.history)
hist_csv_file = 'HISTORY PATH'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

In [ ]:
'''ResNet50'''
def conv_block(inputs, num_filters):
  s = Lambda(lambda x: x / 255) (inputs)
  x = Conv2D(num_filters, 3, padding='same')(s)
  x = BatchNormalization()(x)
  x = Dropout(0.2)(x)
  x = Activation('elu')(x)

  x = Conv2D(num_filters, 3, padding='same')(x)
  x = BatchNormalization()(x)
  x = Dropout(0.25)(x)
  x = Activation('elu')(x)
  
  return x

def decoder(inputs, skip_features, num_filters):
  x = Conv2DTranspose(num_filters, (2,2), strides=2, padding='same')(inputs)
  x = Concatenate()([x, skip_features])
  x = conv_block(x, num_filters)
  return x

def build_resnet50_unit(input_shape, size):
  '''Input'''
  inputs = Input(input_shape)
  '''Pretrained model'''
  resnet50 = ResNet50(include_top = False, weights = 'imagenet', input_tensor = inputs)
  '''encoder'''
  s1 = resnet50.get_layer("input_1").output ##512
  s2 = resnet50.get_layer("conv1_relu").output ##256
  s3 = resnet50.get_layer("conv2_block3_out").output ##128
  s4 = resnet50.get_layer("conv3_block4_out").output ##64
  '''bridge'''
  b1 = resnet50.get_layer("conv4_block6_out").output ##64
  '''decoder'''
  d1 = decoder(b1, s4, int(size/1))
  d2 = decoder(d1, s3, int(size/2))
  d3 = decoder(d2, s2, int(size/4))
  d4 = decoder(d3, s1, int(size/8))
  '''outputs'''
  outputs = Conv2D(3, (1,1), padding='same', activation='softmax')(d4)

  model = Model(inputs, outputs)
  return model


if __name__ == "__main__":
  input_shape = (size, size, 3)
  model = build_resnet50_unit(input_shape, size)
  model.summary()
model.compile(optimizer= opt, loss= iou_loss, metrics = [custom_f1, tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

history_2 = model.fit(train_generator,
                    validation_data = validation_generator,
                    steps_per_epoch=train_generator.n//train_generator.batch_size, 
                    validation_steps = validation_generator.n//validation_generator.batch_size,
                              epochs=epochs, verbose=1)

model.save('SAVE PATH')

results_2 = model.evaluate(x_test, y_test, batch_size=32)
print(results_2)

hist_df = pd.DataFrame(history_2.history)
hist_csv_file = 'HISTORY PATH'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

In [ ]:
'''inceptionv3'''
def conv_block(inputs, num_filters):
  s = Lambda(lambda x: x / 255) (inputs)
  x = Conv2D(num_filters, 3, padding='same')(s)
  x = BatchNormalization()(x)
  x = Dropout(0.2)(x)
  x = Activation('elu')(x)

  x = Conv2D(num_filters, 3, padding='same')(x)
  x = BatchNormalization()(x)
  x = Dropout(0.25)(x)
  x = Activation('elu')(x)
  
  return x

def decoder(inputs, skip_features, num_filters):
  x = Conv2DTranspose(num_filters, (2,2), strides=2, padding='same')(inputs)
  x = Concatenate()([x, skip_features])
  x = conv_block(x, num_filters)
  return x

def build_inceptionv3_unit(input_shape, size):
  '''Input'''
  inputs = Input(input_shape)
  '''Pretrained model'''
  inceptionv3 = InceptionV3(include_top = False, weights = 'imagenet', input_tensor = inputs)
  '''encoder'''
  s1 = inceptionv3.get_layer("input_1").output ##256

  s2 = inceptionv3.get_layer("activation").output ##127
  s2 = ZeroPadding2D(( (1, 0), (1, 0) ))(s2)  #128

  s3 = inceptionv3.get_layer("activation_3").output ##62
  s3 = s3 = ZeroPadding2D((1, 1))(s3)   #64

  s4 = inceptionv3.get_layer("activation_28").output ##29
  s4 = ZeroPadding2D(( (2, 1), (2, 1) ))(s4) 

  '''bridge'''
  b1 = inceptionv3.get_layer("activation_74").output ##14
  b1 = ZeroPadding2D((1, 1))(b1)

  '''decoder'''
  d1 = decoder(b1, s4, int(size/1))
  d2 = decoder(d1, s3, int(size/2))
  d3 = decoder(d2, s2, int(size/4))
  d4 = decoder(d3, s1, int(size/8))
  '''outputs'''
  outputs = Conv2D(3, (1,1), padding='same', activation='softmax')(d4)

  model = Model(inputs, outputs)
  return model
if __name__ == "__main__":
  input_shape = (size, size, 3)
  model = build_inceptionv3_unit(input_shape, size)
  model.summary()
model.compile(optimizer= opt, loss= iou_loss, metrics = [mean_iou, custom_f1, tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

history_5 = model.fit(train_generator,
                    validation_data = validation_generator,
                    steps_per_epoch=train_generator.n//train_generator.batch_size, 
                    validation_steps = validation_generator.n//validation_generator.batch_size,
                              epochs=epochs, verbose=1)

model.save('SAVE PATH')

results_5 = model.evaluate(x_test, y_test, batch_size=32)
print(results_5)

hist_df = pd.DataFrame(history_5.history)
hist_csv_file = 'HISTORY PATH'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)


In [ ]:
model = keras.models.load_model('SAVE PATH', compile=False)

testing = x_test[:200,:,:,:]
y_pred = model.predict(testing)

for i in range(0, len(y_pred)):
  fig, (ax1, ax2, ax3, ax4,ax5, ax6, ax7) = plt.subplots(1, 7, figsize=(22,22))
  ax1.imshow(y_pred[i, :, :, 0], vmin=0, vmax=1)
  ax2.imshow(y_pred[i, :, :, 1], vmin=0, vmax=1)
  ax3.imshow(y_pred[i, :, :, 2], vmin=0, vmax=1)
  ax4.imshow(x_test[i,:,:,:])
  ax5.imshow(y_test[i, :, :, 0], vmin=0, vmax=1)
  ax6.imshow(y_test[i, :, :, 1], vmin=0, vmax=1)
  ax7.imshow(y_test[i, :, :, 2], vmin=0, vmax=1)